In [1]:
try:
    get_ipython().run_line_magic("reset", "-f")
except NameError:
    is_notebook = False
    p = print
else:
    is_notebook = True
    p = display
    # p = print

import warnings

warnings.simplefilter("ignore")

from math import floor
import numpy as np
import pandas as pd

if is_notebook:
    import matplotlib.pyplot as plt
    import seaborn as sns
else:
    import asciichartpy

import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import brier_score_loss
from scipy.stats import norm
from scipy.optimize import minimize

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

input_dir = "march-machine-learning-mania-2025"

In [2]:
df_names = []
df_names_compact = []
df_names_detailed = []

for gender in ["M", "W"]:
    for part in ["RegularSeason", "NCAATourney", "SecondaryTourney"]:
        for result_type in ["Compact", "Detailed"]:
            if part == "SecondaryTourney" and result_type == "Detailed":
                continue
            df_name = f"{gender}{part}{result_type}Results"
            df_names.append(df_name)
            if result_type == "Compact":
                df_names_compact.append(df_name)
            else:
                df_names_detailed.append(df_name)
            print(df_name)
            path = f"../input/{input_dir}/{df_name}.csv"
            df = pd.read_csv(path)
            df["Part"] = part
            globals()[df_name] = df

season_min = WRegularSeasonDetailedResults["Season"].min()

for df_name in df_names:
    df = globals()[df_name]
    globals()[df_name] = df[df["Season"] >= season_min]

MRegularSeasonCompactResults
MRegularSeasonDetailedResults
MNCAATourneyCompactResults
MNCAATourneyDetailedResults
MSecondaryTourneyCompactResults
WRegularSeasonCompactResults
WRegularSeasonDetailedResults
WNCAATourneyCompactResults
WNCAATourneyDetailedResults
WSecondaryTourneyCompactResults


In [ ]:
game_stats_1_2 = pd.DataFrame()

for df_name in df_names_detailed:
    game_stats_1_2 = pd.concat([game_stats_1_2, globals()[df_name]])

game_stats_1_2 = game_stats_1_2.reset_index(drop=True)
mask = game_stats_1_2["WTeamID"] < game_stats_1_2["LTeamID"]


def to_1_2(col_W_L):
    game_stats_1_2.loc[mask, f"{col_W_L}_1"] = game_stats_1_2[f"W{col_W_L}"]
    game_stats_1_2.loc[mask, f"{col_W_L}_2"] = game_stats_1_2[f"L{col_W_L}"]
    game_stats_1_2.loc[~mask, f"{col_W_L}_1"] = game_stats_1_2[f"L{col_W_L}"]
    game_stats_1_2.loc[~mask, f"{col_W_L}_2"] = game_stats_1_2[f"W{col_W_L}"]
    game_stats_1_2[f"{col_W_L}_1"] = game_stats_1_2[f"{col_W_L}_1"].astype(
        game_stats_1_2[f"W{col_W_L}"].dtype
    )
    game_stats_1_2[f"{col_W_L}_2"] = game_stats_1_2[f"{col_W_L}_2"].astype(
        game_stats_1_2[f"W{col_W_L}"].dtype
    )


game_stats_1_2["WLoc"] = game_stats_1_2["WLoc"].astype("category")
game_stats_1_2["LLoc"] = game_stats_1_2["WLoc"].cat.rename_categories(
    {"A": "H", "H": "A"}
)

for c in game_stats_1_2.columns:
    if c.startswith("W"):
        to_1_2(c[1:])

game_stats_1_2["Spread_1"] = game_stats_1_2["Score_1"] - game_stats_1_2["Score_2"]
game_stats_1_2["Spread_2"] = game_stats_1_2["Score_2"] - game_stats_1_2["Score_1"]
game_stats_1_2 = game_stats_1_2[[c for c in game_stats_1_2 if c[0] not in ("W", "L")]]
game_stats_1_2 = game_stats_1_2.drop(columns="NumOT")
game_stats_1_2.insert(2, "TeamID_1", game_stats_1_2.pop("TeamID_1"))
game_stats_1_2.insert(3, "TeamID_2", game_stats_1_2.pop("TeamID_2"))
game_stats_1_2 = game_stats_1_2.sort_values(
    ["Season", "DayNum", "TeamID_1", "TeamID_2"]
)
game_stats_1_2 = game_stats_1_2.reset_index(drop=True)
print(f"game_stats_1_2     {game_stats_1_2.shape[0]:,}")
p(game_stats_1_2)
print()

print(f"MRegularSeasonDetai{MRegularSeasonDetailedResults.shape[0]:>7,}")
print(f"WRegularSeasonDetai{WRegularSeasonDetailedResults.shape[0]:>7,}")
print(
    f"RegularSeasonDetail{MRegularSeasonDetailedResults.shape[0]+WRegularSeasonDetailedResults.shape[0]:>7,}"
)

reg_season = game_stats_1_2[game_stats_1_2["Part"] == "RegularSeason"]
print(f"reg_season         {reg_season.shape[0]:,}")

game_stats_1_o = reg_season.rename(
    columns={c: f"{c[:-2]}_o" for c in reg_season if c[-2:] == "_1"}
)
game_stats_1_o = game_stats_1_o.rename(
    columns={c: f"{c[:-2]}_d" for c in game_stats_1_o if c[-2:] == "_2"}
)
print(f"game_stats_1_o     {game_stats_1_o.shape[0]:,}")

game_stats_2_o = reg_season.rename(
    columns={c: f"{c[:-2]}_o" for c in reg_season if c[-2:] == "_2"}
)
game_stats_2_o = game_stats_2_o.rename(
    columns={c: f"{c[:-2]}_d" for c in game_stats_2_o if c[-2:] == "_1"}
)
print(f"game_stats_2_o     {game_stats_2_o.shape[0]:,}")
print(f"game_stats_1_o+2_o {game_stats_1_o.shape[0]+game_stats_2_o.shape[0]:,}\n")

game_stats_o_d = pd.concat([game_stats_1_o, game_stats_2_o])
game_stats_o_d = game_stats_o_d.drop(columns="Part")
game_stats_o_d.insert(2, "TeamID_o", game_stats_o_d.pop("TeamID_o"))
game_stats_o_d.insert(3, "TeamID_d", game_stats_o_d.pop("TeamID_d"))
game_stats_o_d = game_stats_o_d.sort_values(
    ["Season", "DayNum", "TeamID_o", "TeamID_d"]
)
game_stats_o_d = game_stats_o_d.reset_index(drop=True)
print(f"game_stats_o_d     {game_stats_o_d.shape[0]:,}")
p(game_stats_o_d)
print()

game_stats_1_2     166,128


,Season,DayNum,TeamID_1,TeamID_2,Part,Score_1,Score_2,FGM_1,FGM_2,FGA_1,FGA_2,FGM3_1,FGM3_2,FGA3_1,FGA3_2,FTM_1,FTM_2,FTA_1,FTA_2,OR_1,OR_2,DR_1,DR_2,Ast_1,Ast_2,TO_1,TO_2,Stl_1,Stl_2,Blk_1,Blk_2,PF_1,PF_2,Spread_1,Spread_2
0,2010,7,1107,1393,RegularSeason,43,75,15,29,55,60,5,2,28,17,8,15,14,31,12,14,27,32,10,21,32,21,12,21,4,6,27,16,-32,32
1,2010,7,1108,1326,RegularSeason,60,100,21,39,61,73,7,14,17,33,11,8,20,12,11,13,24,34,7,29,13,6,4,8,5,6,16,19,-40,40
2,2010,7,1143,1293,RegularSeason,75,70,24,26,52,52,5,8,12,21,22,10,32,15,13,11,19,20,15,11,12,17,8,7,1,3,14,24,5,-5
3,2010,7,1198,1314,RegularSeason,72,88,25,34,68,61,8,4,23,13,14,16,17,19,13,12,16,32,15,23,20,26,14,10,2,8,18,19,-16,16
4,2010,9,1143,1178,RegularSeason,95,61,29,20,61,57,7,4,17,18,30,17,35,27,15,14,30,18,11,14,10,15,5,4,3,2,20,21,34,-34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166123,2025,106,3150,3378,RegularSeason,52,70,18,25,58,59,5,4,17,17,11,16,14,18,7,8,24,30,5,15,15,12,8,9,2,3,15,16,-18,18
166124,2025,106,3194,3349,RegularSeason,39,72,10,30,55,63,3,10,17,24,16,2,23,4,4,5,25,39,5,21,10,10,4,6,3,5,9,14,-33,33
166125,2025,106,3242,3416,RegularSeason,63,58,17,23,46,63,4,7,17,26,25,5,28,11,5,15,26,21,9,11,12,12,4,4,5,2,16,21,5,-5
166126,2025,106,3329,3428,RegularSeason,68,64,23,25,63,58,6,5,23,23,16,9,18,16,18,8,25,18,9,16,17,11,6,8,3,6,21,20,4,-4



MRegularSeasonDetai 83,674
WRegularSeasonDetai 80,626
RegularSeasonDetail164,300
reg_season         164,300
game_stats_1_o     164,300
game_stats_2_o     164,300
game_stats_1_o+2_o 328,600



In [ ]:
# Look at the distribution of games per team
games_per_team = game_stats_o_d.rename(columns={"TeamID_o": "TeamID"}).drop(
    columns="TeamID_d"
)
games_per_team_count = games_per_team.groupby(["Season", "TeamID"]).size()

# Count how many team-seasons have each number of games
games_count_distribution = games_per_team_count.value_counts().sort_index()
print("Distribution of games per team-season:")
print(games_count_distribution)

# Look at specific examples of teams with very few games
teams_with_few_games = games_per_team_count[games_per_team_count <= 10].reset_index()
print("\nSample of teams with 10 or fewer games:")
print(teams_with_few_games.head(10))

# Check if there's a pattern by season
few_games_by_season = teams_with_few_games.groupby("Season").size()
print("\nNumber of teams with few games by season:")
print(few_games_by_season)

In [ ]:
# p(game_stats_o_d)
season_stats = game_stats_o_d.rename(columns={"TeamID_o": "TeamID"})
season_stats = season_stats.drop(columns="TeamID_d")
season_stats_g = season_stats.groupby(["Season", "TeamID"])

season_stats = season_stats_g[
    [c for c in season_stats if c.endswith("_o")]
    + [c for c in season_stats if c.endswith("_d")]
].mean()

season_stats = season_stats.reset_index()
season_stats = season_stats.sort_values(["Season", "TeamID"])
season_stats = season_stats.reset_index(drop=True)

print(f"season_stats   {season_stats.shape[0]:,}")
p(season_stats)
print()

In [ ]:
p(season_stats[(season_stats["Season"] == 2010) & (season_stats["TeamID"] == 1393)])
# p(game_stats_o_d[(game_stats_o_d["Season"]==2010) & (game_stats_o_d["TeamID_o"]==1393)])
# p(MRegularSeasonDetailedResults[(MRegularSeasonDetailedResults["Season"]==2010) & (MRegularSeasonDetailedResults["WTeamID"]==1393)])
# p(MRegularSeasonDetailedResults[(MRegularSeasonDetailedResults["Season"]==2010) & (MRegularSeasonDetailedResults["LTeamID"]==1393)])

In [ ]:
matchups = game_stats_o_d[["Season", "DayNum", "TeamID_o", "TeamID_d"]]
p(matchups)

# matchups with the season stats of d
opp_season_stats = pd.merge(
    matchups,
    season_stats,
    left_on=["Season", "TeamID_d"],
    right_on=["Season", "TeamID"],
)
opp_season_stats = opp_season_stats.drop(columns="TeamID")
opp_season_stats = opp_season_stats.sort_values(
    ["Season", "DayNum", "TeamID_o", "TeamID_d"]
)
opp_season_stats = opp_season_stats.reset_index(drop=True)
p(opp_season_stats)

# strength of schedule
sos_stats = opp_season_stats.rename(columns={"TeamID_o": "TeamID"})
sos_stats = sos_stats.drop(columns="TeamID_d")
sos_stats_g = sos_stats.groupby(["Season", "TeamID"])

sos_stats = sos_stats_g[
    [c for c in sos_stats if c.endswith("_o")]
    + [c for c in sos_stats if c.endswith("_d")]
].mean()

sos_stats = sos_stats.reset_index()
sos_stats = sos_stats.sort_values(["Season", "TeamID"])
sos_stats = sos_stats.reset_index(drop=True)

print(f"sos_stats   {sos_stats.shape[0]:,}")
p(sos_stats)
print()

In [ ]:
season_self_sos = pd.merge(
    season_stats, sos_stats, on=["Season", "TeamID"], suffixes=["_self", "_sos"]
)
print(f"season_self_sos {season_self_sos.shape[0]:,}")
p(season_self_sos)
print()

In [ ]:
train = (
    game_stats_1_2.groupby(["Season", "TeamID_1", "TeamID_2"])["Spread_1"]
    .mean()
    .reset_index()
)

train = pd.merge(
    train,
    season_self_sos,
    left_on=["Season", "TeamID_1"],
    right_on=["Season", "TeamID"],
)
train = train.drop(columns="TeamID")

train = train.rename(columns={c: f"{c}_1" for c in train if c[-4:] in ("self", "_sos")})

train = pd.merge(
    train,
    season_self_sos,
    left_on=["Season", "TeamID_2"],
    right_on=["Season", "TeamID"],
)
train = train.drop(columns="TeamID")

train = train.rename(columns={c: f"{c}_2" for c in train if c[-4:] in ("self", "_sos")})

train = train.drop(columns=[c for c in train if c.startswith("Spread_d_")])

train = train.sort_values(["Season", "TeamID_1", "TeamID_2"])
train = train.reset_index(drop=True)

print(f"train        {train.shape[0]:,}")
p(train)
print()

X = train.drop(columns=["Season", "TeamID_1", "TeamID_2", "Spread_1"])

print(f"X        {X.shape[0]:,}")
p(X)
print()

y = train["Spread_1"].rename("y")

print(f"y        {y.shape[0]:,}")
p(y)
print()

In [ ]:
p(train[train["Season"] == 2021])

Missing
- normalized count of home games, away games
- season average of NumOT

In [ ]:
kfold = KFold(shuffle=True, random_state=42)
fold_models = []
y_pred_oof = np.zeros(len(train))

for fold_n, (i_fold, i_oof) in enumerate(kfold.split(train.index)):
    print(f"fold {fold_n}", flush=True)
    m = xgb.XGBRegressor()
    m.fit(X.iloc[i_fold], y.iloc[i_fold])
    fold_models.append(m)
    y_pred_oof[i_oof] = m.predict(X.iloc[i_oof])

In [ ]:
suptitle = "NCAA D1 basketball 2010-2025 men's and women's"
title = "Predicted point spread distribution"

if is_notebook:
    sns.histplot(y_pred_oof, bins=50)
    plt.suptitle(suptitle)
    plt.title(title)

else:
    print()
    print(suptitle)
    print(title)
    hist, edges = np.histogram(y_pred_oof, bins=50)
    width = 80
    print(asciichartpy.plot(hist, {"height": 10, "width": width}))
    w50 = (width - 11) // 2
    w66 = (width - 11) * 2 // 3 - w50
    print(
        f"{floor(edges[len(edges)//2-1]):>{w50}}"
        f"{floor(edges[len(edges)*2//3]):>{w66}}"
    )
    print()

In [ ]:
y_pred_prob = 1 / (1 + np.exp(-y_pred_oof * 0.25))
y_true = (train["Spread_1"] > 0).astype(int)
brier_score = np.mean((y_pred_prob - y_true) ** 2)
print(f"Brier score: {brier_score:.4f}")

claude.ai:

> A Brier score of 0.1479 represents the mean squared error between your predicted win probabilities and the actual outcomes. Let me help you interpret this result:
> 
> #### Understanding the Brier Score
> 
> The Brier score ranges from 0 to 1, where:
> 
> - 0 is perfect prediction (you predicted exactly the right probability for every game)
> - 0.25 represents the score you'd get by always predicting 50% probability (random guessing)
> - 1 is the worst possible score (you predicted with complete confidence the opposite of what happened)
> 
> Your score of 0.1479 is significantly better than random guessing (0.25), which indicates your model has meaningful predictive power.
> 
> #### Contextualizing Your Score
> 
> For NCAA basketball game predictions:
> - Professional betting markets and sophisticated models typically achieve Brier scores in the 0.12-0.16 range
> - Your score of 0.1479 falls within this competitive range
> - The best models in previous March Machine Learning Mania competitions have achieved scores around 0.12-0.13
> 
> #### What This Means
>
> Your model is capturing meaningful patterns in the data. The 0.1479 Brier score suggests that:
> - Your approach of using team season statistics to predict point spreads, then converting to win probabilities, is valid
> - The model has good discriminative power (distinguishing between likely wins and losses)
> - The probability calibration (using the sigmoid with scaling factor 0.25) is reasonably effective

Note: There is data leakage in my approach, which artificially increases the out-of-fold cross-validation scores over scores involving real data (future games.) Specifically, the data leakage is that for regular season games, the season stats in X (the features used to fit/train the model) include the game being predicted. I am accepting this trade-off because it is a simpler, more efficient approach that gets more rows in my train dataset (vs. only predicting hitorical tournament games.) I could remove the game being predicted from the season stats for each row in the train dataset, but that would take more work with potentially little benefit and could greatly increase run time.

In [ ]:
suptitle = "NCAA D1 basketball 2010-2025 men's and women's"
title = "Predicted win probabilities distribution"

if is_notebook:
    plt.figure(figsize=(10, 6))
    sns.histplot(y_pred_prob, bins=50, kde=True)
    plt.axvline(0.5, color="red", linestyle="--", alpha=0.7, label="50% threshold")
    plt.xlabel("Predicted Win Probability")
    plt.ylabel("Count")
    plt.suptitle(suptitle)
    plt.title(title)
    plt.legend()
    plt.show()
else:
    print()
    print(suptitle)
    print(title)
    hist, _ = np.histogram(y_pred_prob, bins=50)
    print(asciichartpy.plot(hist, {"height": 10, "width": 80}))
    print()

claude.ai:

> The distribution actually reveals something important about your modeling approach. It's not centered around 0.5, and that's very interesting!
The bimodal (two-peaked) distribution you're seeing with high peaks near 0 and 1 tells us that your sigmoid transformation is creating very confident predictions. The scaling factor of 0.25 in your sigmoid function is making the transformation quite steep, which pushes many probabilities toward the extremes.
This pattern indicates:
> 
> Your model is making strong, polarized predictions rather than cautious ones hovering around 0.5.
The point spread predictions from your model are being transformed in a way that expresses high confidence - most games are predicted as either strong wins or strong losses.
Since your Brier score is 0.1479 (which is quite good), many of these confident predictions are actually correct! If they weren't, your Brier score would be much worse.
> 
> The distribution doesn't need to be centered at 0.5. In fact, if your model has good discriminative power, and games truly do have clear favorites and underdogs, you would expect to see more predictions at the extremes than in the middle.

In [ ]:
print("Preparing submission")
sample_sub = pd.read_csv(f"../input/{input_dir}/SampleSubmissionStage1.csv")

sample_sub[["Season", "Team1", "Team2"]] = (
    sample_sub["ID"].str.split("_", expand=True).astype(int)
)

team_data = season_self_sos[season_self_sos["Season"] == 2025].set_index("TeamID")
X_submit = pd.DataFrame(index=sample_sub.index)

for col in X.columns:
    parts = col.split("_")
    team_num = parts[-1]
    base_col = "_".join(parts[:-1])
    team_id_col = f"Team{team_num}"
    X_submit[col] = sample_sub[team_id_col].map(team_data[base_col])

ensemble_preds = np.mean([model.predict(X_submit) for model in fold_models], axis=0)
win_probs = 1 / (1 + np.exp(-ensemble_preds * 0.25))
sample_sub["Pred"] = win_probs
sample_sub[["ID", "Pred"]].to_csv("submission.csv", index=False)
print("Wrote submission.csv")

In [ ]:
print("Generating multiple independent simulation-based submissions...")
base_ids = sample_sub["ID"].copy()
probabilities = sample_sub["Pred"].values
num_matchups = len(probabilities)

for num_sims in range(1, 11):
    random_values = np.random.random((num_sims, num_matchups))
    sim_results = (random_values < probabilities).astype(int)
    submission = pd.DataFrame(
        {"ID": base_ids, "Pred": sim_results.sum(axis=0) / num_sims}
    )
    filename = f"submission{num_sims}.csv"
    submission.to_csv(filename, index=False)
    print(
        f"Wrote {filename:>16}, simulation rounds: {num_sims:>2}, win probabilities: {[f"{p:.2f}" for p in sorted(np.unique(submission['Pred']))]}"
    )

print("All independent simulation-based submissions complete!")